This example simply loads a text as 'context' of something that didn't happen yet when the llm was trained.  It then demonstrated 'memory' using the langchain ConverstationChain with memory.

Sample question series that needs 'memory'

Who discovered America?
When was it discovered?

In [8]:
from langchain.llms import OpenAI 
from dotenv import load_dotenv
from pathlib import Path
import os
dotenv_path = Path('.env_secure')
load_dotenv(dotenv_path=dotenv_path)
open_api_key = os.environ.get('OPENAI_API_KEY')


In [9]:
text = ["""
CNN
 — 
A record-breaking Norwegian climber has hit out at what she calls “misinformation and hatred” surrounding claims she and her team climbed over a dying porter on K2 to summit the deadly peak.

Last month, Kristin Harila and Nepali Tenjin Sherpa – known as Lama – smashed the record for the fastest summit of all 14 of the world’s 8,000-meter (26,000-feet) mountains. They completed the feat in three months and one day after climbing Pakistan’s K2, the last peak on their quest and considered to be more technically challenging than Mount Everest.

But their achievement has now been overshadowed by shocking claims that dozens of climbers walked past a Pakistani porter who fell off a sheer edge, was hanging upside down in ropes and later died.

Speaking to CNN on Friday, Harila insisted she and her team did everything they could to save Pakistani mountain porter Mohammad Hassan, and denied that she was in footage circulating of the incident.

Austrian climber Wilhelm Steindl was on the mountain that day, according to an interview he did with the Austrian newspaper The Standard on Tuesday. But he decided to turn back when conditions became too dangerous, he said.

Once back at base camp, he said he watched drone footage of those who continued on the treacherous route, captured by cameraman Philip Flämig. Eyewitnesses later said that they saw a man who had fallen and was left hanging upside down, while climbers passed him on their ascent, according to the report. CNN has not seen the footage.

"""]
source = ["https://www.cnn.com/travel/article/mountaineer-k2-porter-death-scli-intl/index.html"]

langchain memory ==> https://github.com/simpleappdesigner/LangChain_101/blob/main/LangChain_101_memory.ipynb
article ==> https://www.cnn.com/travel/article/mountaineer-k2-porter-death-scli-intl/index.html

In [10]:
from langchain.docstore.document import Document
docs = [Document(page_content="text", metadata={"source": "local"}),]
print(type(docs))
doc_ = [Document(page_content=str(text), metadata={"source": "https://www.cnn.com/travel/article/mountaineer-k2-porter-death-scli-intl/index.html"})]


<class 'list'>


In [12]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
chat = OpenAI()

conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory(),
)

conversation.run("Answer briefly. What are the first 7 letters of the alphabet?")
conversation.run("And the next 4?")

' H, I, J, K.'

In [13]:
from langchain.prompts import PromptTemplate
prompt_template = """You are a Chat customer support agent.\
        Use the following pieces of context to answer the question at the end.\
        If you don't know the answer, just say that you don't know, don't try to make up an answer.\
        {context}\
        Question: {question}\
        Answer: """
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
from langchain.chains.question_answering import load_qa_chain

memory = ConversationBufferMemory(memory_key="chat_history", input_key="question")
chain = load_qa_chain(
    OpenAI(temperature=0), chain_type="stuff", memory=memory, prompt=PROMPT
)

query = "What is Kristin Harila accused of doing on a recent record breaking climb?"
docs = doc_

chain_input={
    "input_documents": docs,
    "context":"This is contextless",
    "question":query,

}
result=chain(chain_input, return_only_outputs=True)


In [14]:
result['output_text']

' Kristin Harila is accused of walking past a Pakistani porter who fell off a sheer edge and was hanging upside down in ropes while she and her team were on their ascent.'

In [15]:
query = "Who was the porter?"
chain_input={
    "input_documents": docs,
    "context":"This is contextless",
    "question":query,

}
result=chain(chain_input, return_only_outputs=True)
result['output_text'].replace("\n", "")

'The porter was Mohammad Hassan.'